# Bases de datos que usas sin saberlo -- Codecamp'18

Este tutorial está disponible en https://github.com/dsevilla/codecamp18.
Diego Sevilla Ruiz, [dsevilla@um.es](mailto:dsevilla@um.es).

In [ ]:
%load utils/functions.py

In [ ]:
from IPython.display import Image
from pprint import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

In [ ]:
yoda(u"Una guerra SQL vs. NoSQL no debes empezar")

In [ ]:
chew("Grrrrr!")

In [ ]:
%%bash
sudo docker pull mongo

In [ ]:
!pip install pymongo
!pip install lxml
!pip install folium
!pip install ipython-cypher

In [ ]:
!sudo docker run --rm -d --name mongo -p 27017:27017 mongo

In [ ]:
import pymongo
from pymongo import MongoClient
client = MongoClient("localhost", 27017)
client

Creamos una base de datos `codecamp18`:

In [ ]:
db = client.codecamp18

Y la colección `participantes18`:

In [ ]:
participantes18 = db.participantes18
participantes18

In [ ]:
participantes18.insert_one(
    {'nombre': 'Diego Sevilla Ruiz',
    'dni' : '12345678X',
    'photo': 'Chewbacca',
    'intereses' : ['charla#2', 'charla#3', 'charla#4', 'charla#5']
    })

In [ ]:
client.database_names()

In [ ]:
DictTable(participantes18.find_one())

In [ ]:
participantes18.insert_one(
    {'nombre': 'María José Ortín Ibáñez',
     'dni' : '23456789Y',
     'intereses' : ['charla#2', 'charla#3', 'charla#4', 'charla#5']
    })

También quieren guardar las diferentes charlas que hay... Para ello, nada mejor que obtener la lista de charlas desde la propia página web... :)

In [ ]:
charlas18 = db.charlas18

In [ ]:
charlas = pd.read_html('https://dafi.inf.um.es/CodeCamp/ediciones/2018/',
                       attrs={'class': 'table'})

In [ ]:
charlas[1]

In [ ]:
for row in charlas[1].itertuples():
    charlas18.insert_one({'_id' : "charla%s" % (row[1]),
                          'título' : row[2],
                          'ponentes' : row[3]
                         })

In [ ]:
ListDictTable(list(charlas18.find()))

# Tipos de Sistemas NoSQL

## MongoDB (documentos)

![Image of MongoDB](https://webassets.mongodb.com/_com_assets/cms/MongoDB-Logo-5c3a7405a85675366beb3a5ec4c032348c390b3f142f5e6dddf1d78e2df5cb5c.png)

Base de datos documental que usaremos como ejemplo. Una de las más extendidas:

- Modelo de documentos JSON (BSON, en binario, usado para eficiencia)
- Map-Reduce para transformaciones de la base de datos y consultas
- Lenguaje propio de manipulación de la base de datos llamado "de agregación" (aggregate)
- Soporta sharding (distribución de partes de la BD en distintos nodos)
- Soporta replicación (copias sincronizadas master-slave en distintos nodos)
- No soporta ACID
- La transacción se realiza a nivel de **DOCUMENTO**

Usaremos `pymongo` desde Python. Para instalarlo:

    sudo pip install --upgrade pymongo


### Texto y título de las diapositivas

Como ya tenemos populada la colección `jisbd17`, podemos actualizar los documentos para añadir el título y el texto de cada diapositiva. Lo extraeremos del fichero `slides.tex`.

In [ ]:
import re

def read_slides():
    in_slide = False
    slidetitle = ''
    slidetext = ''
    slidenum = 0
    with open('slides/slides.tex', 'r') as f:
        for line in f:
            # Remove comments
            line = line.split('%')[0]
            
            if not in_slide:
                if '\\begin{frame}' in line:
                    in_slide = True
            elif '\\frametitle' in line:
                q = re.search('\\\\frametitle{([^}]+)',line)
                slidetitle = q.group(1)
                continue
            elif '\\framebreak' in line or re.match('\\\\only<[^1]',line) or '\\end{frame}' in line:
                
                # Añadir la diapositiva a la lista
                slideid = 'jisbd17-{:03d}'.format(slidenum)
                print(slideid)
                jisbd17.update_one({'_id': slideid},
                       {'$set' : {'title': slidetitle,
                                  'text' : slidetext
                                 }},
                      True)

                # Next
                slidetext = ''
                slidenum += 1
                if '\\end{frame}' in line:
                    in_slide = False
                    slidetitle = ''
            else:
                slidetext += line
                
# Llamar a la función
read_slides()

Para usar el shell de mongo en Javascript:

    docker exec -it mongo mongo

### Consultas sencillas

Distribución del tamaño del texto de las transparencias.

In [ ]:
slides = jisbd17.find(filter={},projection={'text': True})
df = pd.DataFrame([len(s.get('text','')) for s in slides])
df.plot()

La función `find()` tiene un gran número de posibilidades para especificar la búsqueda. Se pueden utilizar cualificadores complejos como:

- `$and`
- `$or`
- `$not`

Estos calificadores unen "objetos", no valores. Por otro lado, hay otros calificadores que se refieren a valores:

- `$lt` (menor)
- `$lte` (menor o igual)
- `$gt` (mayor)
- `$gte` (mayor o igual)
- `$regex` (expresión regular)

In [ ]:
jisbd17.find_one({'text': {'$regex' : '[Mm]ongo'}})['_id']

También permite mostrar el plan de ejecución:

In [ ]:
jisbd17.find({'title' : 'jisbd17-001'}).explain()

### Map-Reduce

Mongodb incluye dos APIs para procesar y buscar documentos: el API de Map-Reduce y el API de agregación. Veremos primero el de Map-Reduce. Manual: https://docs.mongodb.com/manual/aggregation/#map-reduce

![imagen](https://docs.mongodb.com/manual/_images/map-reduce.bakedsvg.svg)

### Histograma de tamaño del texto de las diapositivas

Con Map-Reduce se muestra el tamaño del texto de cada diapositiva, y el número de diapositiva que tienen ese tamaño de texto.

In [ ]:
from bson.code import Code
map = Code(
    '''function () {
           if ('text' in this)
               emit(this.text.length, 1)
           else
               emit(0,1)
       }''')
reduce = Code(
    '''function (key, values) {
            return Array.sum(values);
        }''')
results = jisbd17.map_reduce(map, reduce, "myresults")
results = list(results.find())
results

Como un *plot*:

In [ ]:
df = pd.DataFrame(data = [int(r['value']) for r in results], 
                  index = [int(r['_id']) for r in results], 
                  columns=['posts per length'])
df.plot(kind='bar',figsize=(30,10))

O un histograma:

In [ ]:
df.hist()

### Framework de Agregación

Framework de agregación: https://docs.mongodb.com/manual/reference/operator/aggregation/. Y aquí una presentación interesante sobre el tema: https://www.mongodb.com/presentations/aggregation-framework-0?jmp=docs&_ga=1.223708571.1466850754.1477658152

![agragación](https://docs.mongodb.com/manual/_images/aggregation-pipeline.bakedsvg.svg)

In [ ]:
list(jisbd17.aggregate( [ {'$project' : { 'Id' : 1 }}, {'$limit': 20} ]))

In [ ]:
nposts_by_length = jisbd17.aggregate( [
        #{'$match': { 'text' : {'$regex': 'HBase'}}},
        {'$project': {
            'text' : {'$ifNull' : ['$text', '']}
        }},
        {'$project' : {
             'id' : {'$strLenBytes': '$text'},
             'value' : {'$literal' : 1}
        }
        },
        {'$group' : {
            '_id' : '$id',
            'count' : {'$sum' : '$value'}
        }
        },
        {'$sort' : { '_id' : 1}}
        ])
list(nposts_by_length)

### Simulación de JOIN: `$lookup`

El *framework* de agregación introdujo también una construcción equivalente a `JOIN` de SQL. Por ejemplo, se puede mostrar los títulos de las transparencias referenciadas además de los identificadores:

In [ ]:
list(jisbd17.aggregate( [
        {'$lookup' : {
            "from": "jisbd17",
            "localField": "xref",
            "foreignField": "_id",
            "as": "xrefTitles"
        }},
        {'$project' : {
            '_id' : True,
            'xref' : True,
            'xrefTitles.title' : True
        }}
        ]))

In [ ]:
df = pd.read_csv('extra/municipios-españa-2017.csv.gz',header=0,compression='gzip')

In [ ]:
df.head()

In [ ]:
ciudades = ['Mula']

In [ ]:
mula = df[df.NOMBRE_ACTUAL==ciudades[0]].iloc[0]
mula

In [ ]:
import folium

map = folium.Map(location=[38,-1.5],zoom_start=9)
for nombre_ciudad in ciudades:
    ciudad = df[df.NOMBRE_ACTUAL == nombre_ciudad].iloc[0]
    folium.Marker(location=[ciudad.LATITUD_ETRS89,ciudad.LONGITUD_ETRS89],
                  popup=nombre_ciudad).add_to(map)

map

## Neo4j (Grafos)

![Image of HBase](https://neo4j.com/wp-content/themes/neo4jweb/assets/images/neo4j-logo-2015.png)

Se puede utilizar el propio interfaz de Neo4j también en la dirección http://127.0.0.1:7474.

In [ ]:
%%bash
sudo docker pull neo4j
sudo docker run -d --rm --name neo4j -p 7474:7474 -p 7687:7687 --env NEO4J_AUTH=none neo4j

Vamos a cargar la extensión `ipython-cypher` para poder lanzar consultas Cypher directamente a través de la hoja. He iniciado la imagen de Neo4j sin autenticación, para pruebas locales.

Utilizaremos una extensión de Jupyter Notebook que se llama `ipython-cypher`. Está instalada en la máquina virtual. Si no, se podría instalar con:

    pip install ipython-cypher
    
Después, todas las celdas que comiencen por `%%cypher` y todas las instrucciones Python que comiencen por `%cypher` se enviarán a Neo4j para su interpretación.

### El lenguaje Cypher


### Ipython-cypher

In [ ]:
%load_ext cypher

In [ ]:
%config CypherMagic.auto_html=False
%config CypherMagic.auto_pandas=True

In [ ]:
%%cypher
match (n) return n;

In [ ]:
%%cypher
match (n) return n.name;

Vamos a añadir las relaciones `xref` que haya en las diapositivas. Por ahora sólo había unas puestas a mano. Para las diapositivas que no tengan referencias, añado una al azar.

In [ ]:
import random

nslides = jisbd17.count()
for doc in jisbd17.find():
    for ref in doc.get('xref',['jisbd17-{:03d}'.format(random.randint(1,nslides))]):
        slide_from = doc['_id']
        slide_to = ref
        %cypher MATCH (f:Slide {name: {slide_from}}), (t:Slide {name: {slide_to}}) MERGE (f)-[:REF]->(t)

In [ ]:
%config CypherMagic.auto_networkx=False
%config CypherMagic.auto_pandas=False

In [ ]:
%%cypher
MATCH p=shortestPath(
  (s:Slide {name:"jisbd17-004"})-[*]->(r:Slide {name:"jisbd17-025"})
)
RETURN p

In [ ]:
# Tópicos de slides con expresiones regulares

In [ ]:
import cypher
cypher.run("MATCH (n) RETURN n")

In [ ]:
!sudo docker stop neo4j

In [ ]:
!sudo docker stop mongo